In [ ]:
# Want B to C   removing one of the pairs 

#The Goal is to take a pdb file and identify all the WC base pairs and the hydrogen bonded atoms(?)
# Then create an input file in Plumed's format to define the distances between those hydrogen bonded atoms so
# that a constraints/restraint function can be added later

# This is specifically for CD44 SS_B right now though can be reworked for other uses

# Running Plumed with these intial constraints/restraints will then allow me to reduce the phase space and 
# then be able to reduce computational run time

# Inputs: .pdb file
# Return: some plumed input file (?)

# Made by Isabel Dengos (ijdengos@gmail.com)
# February 2022
# Chen Lab, RNA Institute, UAlbany

In [ ]:
# Import all packages needed
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
import numpy as np
import sys

In [ ]:
# Import all files needed
pdb = PDBFile("/network/rit/home/ID653677/ChenRNALab/idengos/CD44/SS_B/DistRestGro2/conf.pdb")

In [ ]:
#beginig of Plumed output build
k = 0

lines = ['[ distance_restraints ]', 
         '; ai  aj  type  index  type  low  up1  up2  fac']

with open('DistCon.m', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')
        f.close
        
#for checking in pymol to see if okay later
for_pymol = 'select'

In [ ]:
# Use OpenMM to take out the residues?
top = pdb.getTopology()
pos = pdb.getPositions(True)


In [ ]:
#Making lists of residue pairs for the structure /(believe me this was the easist way since 40 was missing)
#Missing top loop piece because it always tries to bond weird
#This has 11 and 35 missing so it can be free to convert to SS_C
left_side = [*range(0,11), *range(17,25)]
right_side = [*range(47,40,-1), *range(39,35,-1,), *range(34,24,-1)]

#Check to make sure your lists are what you need for your particular structure
print(left_side)
print(right_side)

#check that they're the same length
print(len(right_side))


In [ ]:
for r in (0,1):
    
    for t in range(len(left_side)):
        L = left_side[t]
        R = right_side[t]
   
        #Check for the hyrodegens on the leftside, then the rightside
        if r == 0:
            A = L
            B = R
        else:
            B = L
            A = R
        
        resA = list(top.residues())[A]
        atom_idsA = []
        for atom in resA.atoms():
            atom_idsA.append(atom.id)

        resB = list(top.residues())[B]
        atom_idsB = []
        for atom in resB.atoms():
            atom_idsB.append(atom.id)
      
    
    

        first_H = [] 
        n_list = []
        for atom in resA.atoms():
            if atom.element.symbol == 'H':
                n_list.append(atom.id)
                first_H.append(pos[atom.index])
    
        non_H = []
        m_list = []
        for atom in resB.atoms():
            if atom.element.symbol != 'H':
                m_list.append(atom.id)
                non_H.append(pos[atom.index])
        non_Hv=np.array(non_H)

        

        for i in range(len(first_H)):

            first_Hv = np.array(first_H[i])


            dist = np.linalg.norm(non_Hv-first_Hv, axis=1)
            mini = np.min(dist)
            if mini < .25:
                j = np.argmin(dist)
                


                newl = '{:d}  {:d}  1  {:d}  1  0.1  .29  .4  1'.format(int(n_list[i]), int(m_list[j]), k)
                p1newl = ' id {:d} or id {:d} or'.format(int(n_list[i]), int(m_list[j])) #for pymol easy access
                for_pymol += p1newl    

                with open('DistCon.m', 'a') as f:
                        f.write('\n')
                        f.write(newl)
                        k = k+1


In [ ]:

print(for_pymol)

#Should use this to inspect the atoms it collected

# There will be weirdness around the loop at the top; may need to remove them from the plumed file
# by hand based on what it looks like

# I have been doing this by turning the whole structure white; using the selection command to turn all
# the selected atoms a color and the visually inspecting them

# I chose not to automate this removal because when used for other comformations (other than B), it will be 
# different each time